# CPRN CREATE : PHASE A3 : FAC BINDING
**CPRN构建 A3 : 设施绑定**

---

`version` : `4.1.6`   
`author` : seika <seika@live.ca>   
`create_date` : `2025-05-13`   
`last_update` : `2025-10-28`   

---
**A3 阶段功能** ⌼ 设施绑定到基础路网

## ⓿🌞 CONFIGS | 配置项

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd

# --- SETTINGS ---

VERSION_VENDOR = 'V416'    # Version Vendor for Output

## PARAM: CRS
CRS_CGCS2, CRS_WGS84 = 'epsg:4549', 'epsg:4326'

## PARAM: DIST for fac snap/proj
MAX_SNAP_DIST = 500      # 设置最大吸附距离, 单位: 米
MAX_PROJ_DIST = 50       # 设置最大投影距离, 单位: 米

## PARAM: CONSTANT
CONST_HEIGHT = 00.00     # 标准海平面高度 (单位: 米)

## EXTENSIONS
PATH_SQLITE_EXT : str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'

## INPUTS
PATH_SQLITE_HDMAP : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/HDMAP2505.sqlite'
PATH_GPKG_RRL_PATCH : str = r"/Users/seika/sync/lab_works/ZSTORE/geodata/geopackage/hdrrl_patch.gpkg"
PATH_SQLITE_SBDS_ARK : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/ARK_SBDS.sqlite'

### 加载数据表
LAYER_NAME_RRL_PATCH : str = 'CPRN_rrl_patch_v416_jiangsu'                                          # 路网补丁数据(处理后的OSM地图用于补充HDMAP的缺失区域)
LAYER_NAME_CPRN_FACPROJ_SA : str = 'cprn_facproj_sa_dcp_v411_jiangsu'                               # 服务区设施投影顶点(服务区设施嵌入)

## OUTPUTS
PATH_SQLITE_CPRN : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'
PATH_FOLDER_CPRN_PAYLOAD : str = r'/Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/'

### 可计算路网持久化数据表: 
LAYER_NAME_CPRN_FACPROJ_GTR : str = 'CPRN_facproj_gtr_hdmap_' + VERSION_VENDOR + '_jiangsu'                   # 门架到路网投影线 (投影线)
LAYER_NAME_CPRN_FACPROJ_FAILED_GTR : str = 'CPRN_facproj_gtr_failed_hdmap_' + VERSION_VENDOR + '_jiangsu'     # 门架到路网投影线失败 (投影线)

LAYER_NAME_CPRN_ROADS_FACEMBD : str = 'CPRN_roads_facembd_hdmap_' + VERSION_VENDOR + '_jiangsu'               # 门架到路网投影后道路参考线 (路线)
LAYER_NAME_CPRN_ROADS_FACEMBD_SEGMENT : str = 'CPRN_roads_segment_hdmap_' + VERSION_VENDOR + '_jiangsu'       # 门架到路网投影后道路参考线 (路线)
LAYER_NAME_CPRN_ROADS_FACEMBD_SEGMENT_PG : str = 'cprn_road_segment_jiangsu_' + VERSION_VENDOR + '_250910'    # 门架到路网投影后道路参考线 (路线) [写入 PG 库]

LAYER_NAME_CPRN_DG_GDF_HDMAP: str = 'CPRN_dg_roads_hdmap_' + VERSION_VENDOR + '_jiangsu'                      # 可计算路网转GIS路线 (路线/路网)

LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP: str = 'CPRN_dg_divcon_nodes_' + VERSION_VENDOR + '_jiangsu'              # 可计算路网检查项 (分合流点)
LAYER_NAME_CPRN_TOPO_GTR_LVL1_INTVL_HDMAP: str = 'CPRN_topo_gtr_lvl1_intvl_' + VERSION_VENDOR + '_jiangsu'    # 可计算路网应用 (门架拓扑)

LAYER_NAME_CPRN_SHORT_EDGES_GEOM: str = 'CPRN_dg_short_edges_geom_' + VERSION_VENDOR + '_jiangsu'             # 可计算路网检查项 (分合流点)

## OTHERS
TILE_GOOGLE_SATELLITE_URL = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}"

## ㊀ Load CPRN BASE MODEL

In [4]:
import pandas as pd
from cprn.data.pickle import PickleIO

MODELNAME_CPRN_BASE : str = 'CPRN_DG_BASE_JS_FUSIONMAP_V416_251028_77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2.tar.gz'

DG_CPRN = PickleIO.load_from_pickle(PATH_FOLDER_CPRN_PAYLOAD + MODELNAME_CPRN_BASE, compress=True)
print(DG_CPRN)

File hash checking : 77387ff4e6adc9c4e2a2e831d15c37cdd7a58a51a160f9660ea0b760b127e3f2 : passed
DiGraph with 219848 nodes and 223773 edges


## ㊁ FAC BIND to CPRN

设施绑定: 
  - 输入1 : CPRN DG
  - 输入2 : 设施数据 (DF/GDF)
  - 输出 : 绑定后的 CPRN DG
  - 过程: 
    - 根据设施数据的几何信息, 在 CPRN DG 中查找最近邻的边
    - 将设施数据绑定到最近邻的边上
    - 返回绑定后的 CPRN DG
    

### ㊁🅰 Gantry Binding
Bind Gantries to CPRN_DG
- input : `CPRN_DG`
- input : Gantry Data (retrieved from DB, table : `CPRN_facproj_gtr_hdmap_V416_jiangsu`)
- output : `DG_CPRN_FAC_EMBD`

In [5]:
## ㊁🅰① Load Gantry Projection Data from Persisted Table
# 从持久化表读取门架投影数据

from taisl_sop.data.base.spatialite import SpatialitePuller

# 初始化 SQLite 连接
sp_cprn = SpatialitePuller(
    path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT,
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

# 构建 SQL 查询
sql_gtr_proj = f"""
    SELECT fac_code, fac_type, fac_name, dist_snap,
           gh_fac, gh_fac_proj, ghz_fac_proj,
           ASText(GEOMETRY) AS wkt
      FROM {LAYER_NAME_CPRN_FACPROJ_GTR};
      """

print(f"📂 Loading GTR-PROJ from layer/table : {LAYER_NAME_CPRN_FACPROJ_GTR}")

# 使用 query_sql_gdf 方法读取数据
gdf_gtr_proj = sp_cprn.query_sql_gdf(sql_gtr_proj, col_geom=['wkt'])
gdf_gtr_proj.rename_geometry('geom', inplace=True)

print(f"✅ Loaded {len(gdf_gtr_proj)} gantry projection records")
print(f"📋 Columns: {list(gdf_gtr_proj.columns)}")

gdf_gtr_proj.sample(min(4, len(gdf_gtr_proj)))

📂 Loading GTR-PROJ from layer/table : CPRN_facproj_gtr_hdmap_V416_jiangsu
✅ Loaded 2590 gantry projection records
📋 Columns: ['fac_code', 'fac_type', 'fac_name', 'dist_snap', 'gh_fac', 'gh_fac_proj', 'ghz_fac_proj', 'wkt', 'geom']


,fac_code,fac_type,fac_name,dist_snap,gh_fac,gh_fac_proj,ghz_fac_proj,wkt,geom
512,S002832002000520010,G1,扬州西至蜀冈枢纽1,8.861816,WTUB79E451Y1,WTUB79E4142N,WTUB79E4142N002816,LINESTRING Z(119.337753 32.396288 28.160832557...,"LINESTRING Z (119.33775 32.39629 28.16083, 119..."
2563,G000232002000910010,G1,泰兴东枢纽-广陵枢纽1,15.230344,WTTQYKMRNXR0,WTTQYKMRZZTQ,WTTQYKMRZZTQ001433,LINESTRING Z(120.164969 32.14861 14.3324104333...,"LINESTRING Z (120.16497 32.14861 14.33241, 120..."
1858,S002832001040120010,G5,海安南出口匝道,5.316160,WTV8WRW84CBR,WTV8WRQXFCJE,WTV8WRQXFCJE001386,LINESTRING Z(120.518078 32.47285 13.8560036832...,"LINESTRING Z (120.51808 32.47285 13.856, 120.5..."
2583,S006532002000420010,G1,单集至大许枢纽,9.011772,WW5DSF055FCT,WW5DSF04CTSR,WW5DSF04CTSR000000,"LINESTRING Z(117.630778 34.20097 0, 117.630685...","LINESTRING Z (117.63078 34.20097 0, 117.63069 ..."


In [6]:
## PREPARE
cols_facembd = ['fac_code', 'fac_type', 'fac_name', 'gh_fac', 'ghz_fac_proj',] #  'rrl_id', 'osm_id']
gdf_fac_gtr_bind = gdf_gtr_proj[cols_facembd].copy()
gdf_fac_gtr_bind = gdf_fac_gtr_bind.rename(columns={'ghz_fac_proj':'vtx_fac',}) # 'rrl_id':'rrl_id_hdm', 'osm_id':'rrl_id_osm'})
gdf_fac_gtr_bind['fac_ghz'] = gdf_fac_gtr_bind['gh_fac'] + '-00000'
gdf_fac_gtr_bind = gdf_fac_gtr_bind[['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz',]] #'rrl_id_hdm', 'rrl_id_osm',]]

print(f"gtr to bind : {len(gdf_fac_gtr_bind)}")

## BIND
from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN, gdf_fac = gdf_fac_gtr_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

gtr to bind : 2590


In [7]:
# check
# gdf_fac_gtr_bind.sample(4)
DG_CPRN_FAC_EMBD.nodes['WTSW3V30BSGD002220']['lst_fac_attr']

[{'vtx_fac': 'WTSW3V30BSGD002220',
  'fac_code': 'G004232001041510010',
  'fac_type': 'G4',
  'fac_name': '南京收费站入口匝道',
  'fac_ghz': 'WTSW3V310MZG-00000'}]

### ㊁🅱 DivConv Binding
- 从 CPRN 提取分合流点
- 将 分合流点(的属性) 作为设施嵌入到 CPRN

In [8]:
### EXTRACT DC NODE

import pandas as pd
from taisl_sop.model.topo_nx import TopoNx

dg_cprn = DG_CPRN_FAC_EMBD.copy()
dct_dc_nodes = TopoNx.find_divergent_convergent_nodes(dg_cprn)  #TODO: 需要优化

df_dc_nodes = pd.DataFrame(dct_dc_nodes).T.drop(
    columns=['is_divergent', 'is_convergent', 
             'in_edges_attr', 'out_edges_attr', 'node_type'])

def get_dc_type(in_degree, out_degree):
    if in_degree > 1 and out_degree == 1:
        return 'CONV'
    elif in_degree == 1 and out_degree > 1:
        return 'DIV'
    elif in_degree >1 and out_degree > 1:
        return 'MULTI'
    else:
        return 'OTHER'

## make df of dcp fac-like (for further binding)

df_dc_nodes['dc_type'] = df_dc_nodes.apply(
    lambda row: get_dc_type(row['in_degree'], row['out_degree']), axis=1)
df_dc_nodes['gh_dcp'] = df_dc_nodes.index

dct_col_rename_factype = {'CONV': 'DC1', 'DIV': 'DC2', 'MULTI': 'DC3', 'OTHER': 'DCX'}
df_dc_nodes['fac_type'] = df_dc_nodes['dc_type'].map(dct_col_rename_factype)
df_dc_nodes = df_dc_nodes.query("dc_type != 'OTHER'")   # ⚠️ 其他类型不嵌入, e.g., 0->2, 3->0

df_dc_nodes['vtx_fac'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_code'] = df_dc_nodes['gh_dcp']
df_dc_nodes['fac_name'] = df_dc_nodes['in_degree'].astype(str) + '|' + df_dc_nodes['out_degree'].astype(str)

from taisl_sop.model.geohash import Geohash
df_dc_nodes['geom_fac_wgs_3d'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=True)['geometry']
df_dc_nodes['geom_fac_wgs'] = Geohash.decode_df_gdf(df_dc_nodes, colname_geohash='gh_dcp', decode_z=False)['geometry']

print(len(df_dc_nodes))
df_dc_nodes.sample(4)

9113


,in_degree,out_degree,dc_type,gh_dcp,fac_type,vtx_fac,fac_code,fac_name,geom_fac_wgs_3d,geom_fac_wgs
WTV3S1QBN226000903,1,3,DIV,WTV3S1QBN226000903,DC2,WTV3S1QBN226000903,WTV3S1QBN226000903,1|3,POINT Z (120.06813 32.61429 9.03),POINT (120.06813 32.61429)
WW57D094X25W003012,2,1,CONV,WW57D094X25W003012,DC1,WW57D094X25W003012,WW57D094X25W003012,2|1,POINT Z (117.15989 34.36841 30.12),POINT (117.15989 34.36841)
WTWJBYVC57RV001381,1,3,DIV,WTWJBYVC57RV001381,DC2,WTWJBYVC57RV001381,WTWJBYVC57RV001381,1|3,POINT Z (120.97851 31.98551 13.81),POINT (120.97851 31.98551)
WWH3KE44GVD7000946,1,2,DIV,WWH3KE44GVD7000946,DC2,WWH3KE44GVD7000946,WWH3KE44GVD7000946,1|2,POINT Z (118.67723 33.98671 9.46),POINT (118.67723 33.98671)


In [9]:
## BIND DCP to CPRN
cols_fac_embd_dcp = ['vtx_fac', 'fac_code', 'fac_type', 'fac_name',]
df_fac_dcp_bind = df_dc_nodes[cols_fac_embd_dcp]

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN_FAC_EMBD, gdf_fac = df_fac_dcp_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

#### 🌖 (支线任务) 分/合流点 导出地图数据检查

In [10]:
subtask_export_dcp_map = False

if subtask_export_dcp_map:
    cols_fac_dcp = ['fac_code', 'fac_type', 'fac_name', 'vtx_fac', 'geom_fac_wgs', 'geom_fac_wgs_3d',]
    gdf_dc_nodes = gpd.GeoDataFrame(df_dc_nodes[cols_fac_dcp])
    gdf_dc_nodes = gdf_dc_nodes.set_geometry('geom_fac_wgs_3d')
    gdf_dc_nodes = gdf_dc_nodes.drop(columns='geom_fac_wgs')

    print(LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP)

    #🌖 EXPORT TO DB
    gdf_dc_nodes.to_file(PATH_SQLITE_CPRN, driver='SQLite', spatialite=True, 
                         layer=LAYER_NAME_CPRN_CHECK_DCNODES_HDMAP, overwrite=True,
                         index=False, mode='w', encoding='utf-8')

### ㊁🅲 SA(DC) Binding
从 DB 中获取 SA_DCP 数据, 由过程 B1 `PHASE_B1_FAC_PROC_SA.ipynb` 生成


In [14]:
# Fetch SA_DCP data from DB, check `PHASE_B1_FAC_PROC_SA_V416.ipynb` for how to generate it.

LAYER_NAME_CPRN_FACPROJ_SA = 'cprn_facproj_sa_dcp_v416_jiangsu'

from taisl_sop.data.cprn.fac.fac_bind_sa import FacBindSaFetcher
sp_cprn = SpatialitePuller(path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT, 
    set_precision=True, decimal_precision=16, driver='sqlite', verbose=False)

fac_bind_sa_fetcher = FacBindSaFetcher(spatialite_puller=sp_cprn, 
    tb_name=LAYER_NAME_CPRN_FACPROJ_SA, verbose=False)

gdf_fac_sa_dcp = fac_bind_sa_fetcher.fetch_fac_bind_sa().drop(columns=['wkt', 'fac_bind_geohash', 'fac_geohash'])
gdf_fac_sa_dcp = gdf_fac_sa_dcp.rename(columns={'fac_vtx':'vtx_fac'})
gdf_fac_sa_dcp

,fac_code,fac_name,fac_type,vtx_fac,fac_ghz,geom
0,S0019A10000XT0N,西亭服务区(北),SA1,WTWNFCZPYQH4001787,WTWNFFMP4210-00000,"LINESTRING Z (121.06531 32.13758 0, 121.06824 ..."
1,S0019A10000XT0S,西亭服务区(南),SA1,WTWNFF9V3YXW001786,WTWNFF5YDWXK-00000,"LINESTRING Z (121.06362 32.13616 0, 121.06084 ..."
2,G0002A10000GL0W,广陵服务区(西),SA1,WTTQXTTPBD5V001400,WTTQXTHW7ECP-00000,"LINESTRING Z (120.21873 32.10864 0, 120.2193 3..."
3,G0025A10000TH0E,宜兴太湖服务区(东),SA1,WTT32W0X6GS1001290,WTT32WC2FDW1-00000,"LINESTRING Z (119.90662 31.19445 0, 119.9056 3..."
4,G0015A1000HGS0E,花果山服务区(东),SA1,WWHTJ8WBPU5U000780,WWHTJ8ZXGRWT-00000,"LINESTRING Z (119.08027 34.6344 0, 119.07943 3..."
...,...,...,...,...,...,...
477,S0035A10TZDQN0S,泰大桥服务区(南),SA1,WTTPJEW4Q9SB001324,WTTPJGC8ENQJ-00000,"LINESTRING Z (119.78613 32.18869 0, 119.78147 ..."
478,S0018A10000JP0N,界牌服务区(北),SA1,WWJ601DVNMVK000000,WWJ4PCWGXXKF-00000,"LINESTRING Z (119.88142 34.11044 0, 119.88686 ..."
479,S0018A10000JP0N,界牌服务区(北),SA2,WWJ4P2PSK7ZX000000,WWJ4PCWGXXKF-00000,"LINESTRING Z (119.88142 34.11044 0, 119.86035 ..."
480,S0018A10000JP0S,界牌服务区(南),SA1,WWJ4P2PEY5HK000000,WWJ4PCRHVHJD-00000,"LINESTRING Z (119.88166 34.10927 0, 119.86042 ..."


In [15]:
# BIND SA_DCP to CPRN
cols_fac_sa_dcp = ['vtx_fac', 'fac_code', 'fac_type', 'fac_name', 'fac_ghz']
gdf_fac_sa_dcp_bind = gdf_fac_sa_dcp[cols_fac_sa_dcp]

from taisl_sop.model.cprn.fac_embd import FacEmbedder as FacEmbd
DG_CPRN_FAC_EMBD = FacEmbd.fac_embed_cprn(cprn = DG_CPRN_FAC_EMBD, gdf_fac = gdf_fac_sa_dcp_bind, 
    col_vtx = 'vtx_fac', inplace = False, verbose = True)

In [16]:
print(DG_CPRN_FAC_EMBD)

DiGraph with 219848 nodes and 223773 edges


### 五🄳 枢纽绑定 

☄️ (待开展) TODO TASK 

### ㊄🅇 从CPRN取回BindedFAC检查

In [17]:
from cprn.model.topo.topo_search import CprnTopoSearch as CTS

df = CTS.list_fac_df(DG_CPRN_FAC_EMBD)

print(f" fetch embede fac from CPRN: {len(df)}" )
df.sample(4) 

 fetch embede fac from CPRN: 12185


,vtx_fac,fac_code,fac_type,fac_name,fac_ghz
11463,WTVHT31EJF0Z000936,WTVHT31EJF0Z000936,DC2,1|2,NaN
3631,WTWT84NEJNNQ001629,S002832004000210010,G1,南阳至合作1,WTWT84NER4ZW-00000
1023,WTST5B72V96F001178,WTST5B72V96F001178,DC1,2|1,NaN
1314,WTTF9UY9YZMP001304,WTTF9UY9YZMP001304,DC1,2|1,NaN


In [21]:
df.query(" fac_name == '八卦洲服务区(东)' ")

,vtx_fac,fac_code,fac_type,fac_name,fac_ghz
1977,WTSX009UVD3P001942,G0036A1000BGZ0E,SA1,八卦洲服务区(东),WTSX012JSYMW-00000
2518,WTSRPCYPSBYR000960,G0036A1000BGZ0E,SA2,八卦洲服务区(东),WTSX012JSYMW-00000


## ㊅ PERSISTENCE 1: 设施嵌 CPRN 持久化 (PICKLE)

In [22]:
from cprn.data.pickle import PickleIO

CPRN_MODEL_NAME = r'CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V416.pkl'
PickleIO.dump_as_pickle(DG_CPRN_FAC_EMBD, PATH_FOLDER_CPRN_PAYLOAD + CPRN_MODEL_NAME, compress=True)

File renamed from /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V416_251029_temp.tar.gz to /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V416_251029_eaaae2340180fea006386a1cb224a48c6f08f502da707736b97f22826b2fe43d.tar.gz
Compressed pickle file saved as: /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_FAC_EMBD_JS_FUSIONMAP_V416_251029_eaaae2340180fea006386a1cb224a48c6f08f502da707736b97f22826b2fe43d.tar.gz


------

In [16]:
print("EOF")

EOF
